In [17]:
import pandas as pd

TRAIN_DATA = "C:\\Users\\X1\\Downloads\\digit-recognizer\\train.csv"
TEST_DATA = "C:\\Users\\X1\\Downloads\\digit-recognizer\\test.csv"
################### Load train data ####################
dfTrain = pd.read_csv(TRAIN_DATA)
Y_train = dfTrain[['label']]
X_train = dfTrain.drop(["label"],axis=1)


In [18]:
####################### Load test data #################
dfTest = pd.read_csv(TEST_DATA)
X_test = dfTest
XX =X_train
X_test.shape

(28000, 784)

In [19]:
#from sklearn.preprocessing import Imputer as imp
########################## Standarisation #######################################
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def min_normalization(xmat):
    minn = xmat.min()
    return xmat / minn

def max_normalization(xmat):
    maxx = xmat.max()
    return xmat / maxx


def mean_normalization(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat
    
def zScore_Standarisation(xmat):
    ssX = StandardScaler()
    xmat = ssX.fit_transform(xmat)
    return xmat

In [20]:
########################### chi2 TEST ##########################
from sklearn.feature_selection import SelectKBest, chi2
def chi_square(X, y,nbf=100):
    ch2 = SelectKBest(chi2, k=nbf)
    my_transformed_matrix = ch2.fit_transform(X,y)
    return my_transformed_matrix

In [21]:
####################### LOG transformation ##################
import numpy as np
def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)   

In [22]:
################################# sklearn - relief ##############
import sklearn_relief as relief
def apply_relief_selection(X,y,nbf=100):
    r = relief.Relief(n_features=nbf) 
    my_transformed_matrix = r.fit_transform(X,y)
    return my_transformed_matrix


In [23]:
######################### wrraper RFE feature selection ###########
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
def apply_rfe(X,y,threshold = 5):
    global XX
    model_rf = RandomForestClassifier(n_estimators=500, random_state=0, max_depth = 3)
    selector = RFE(model_rf, n_features_to_select=threshold, step=1) 
    selector = selector.fit(X, y)
    selector_ind = selector.get_support()
    my_transformed_matrix = XX.iloc[:, selector_ind].values
    return my_transformed_matrix


In [24]:
############################ embedded methode feature selection ####
from sklearn.feature_selection import SelectFromModel
def embedded_selection(x,y):
    model_rf = RandomForestClassifier(n_estimators=500, random_state=0, max_depth = 3) 
    model_rf.fit(x, y)
    global XX
    sel_sfm = SelectFromModel(model_rf, prefit=True)
    sel_sfm_index = sel_sfm.get_support()
    my_transformed_matrix =  XX.iloc[:, sel_sfm_index].values
    return my_transformed_matrix

In [25]:
################################## PCA ##########################################
from sklearn.decomposition import PCA
def applyPCA(matt,p = 0.95):
    pca = PCA(n_components = p)
    pca.fit(matt)
    matt = pca.transform(matt)
    return matt

In [26]:
######################### Split train+test #######################################
from sklearn.model_selection import train_test_split
def split_data(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20,random_state=42)
    return (X_train, X_test, y_train, y_test)


In [27]:
from time import time
from sklearn.svm import SVC
def train_predict(X_train, X_test, y_train, y_test,typee):
    print("#####################  ", typee ," #########################")
    svclassifier = SVC(kernel='rbf') #0.9591836734693877
    t0 = time()
    svclassifier.fit(X_train, y_train) 

    y_pred = svclassifier.predict(X_test)
    t1 = time()
    ######################## Evaluating the Algorithm ########################
    from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("SVM accueacy : ",accuracy_score(y_test,y_pred))
    print("Time : ",t1 - t0)
    print("#########################################################")

In [13]:
X_tr, X_t, y_tr, y_t = split_data(X_train,Y_train)
train_predict(X_tr, X_t, y_tr, y_t," raw data ") ### row data without any feature selection 

#####################    raw data   #########################
[[807   0   0   0   2   1   4   0   1   1]
 [  0 902   1   1   1   1   0   1   1   1]
 [  3   3 820   2   8   0   3   3   4   0]
 [  1   3   5 898   0   9   0   6  11   4]
 [  1   0   1   0 815   1   5   3   0  13]
 [  1   1   1   8   2 679   6   0   2   2]
 [  3   1   0   0   0   1 777   0   3   0]
 [  0   2   7   0   4   1   0 867   2  10]
 [  1   2   3   7   5   4   0   2 810   1]
 [  2   2   1  10  10   1   0   7   3 802]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.98      0.99      0.99       909
           2       0.98      0.97      0.97       846
           3       0.97      0.96      0.96       937
           4       0.96      0.97      0.97       839
           5       0.97      0.97      0.97       702
           6       0.98      0.99      0.98       785
           7       0.98      0.97      0.97       893
           8       0

In [14]:
X_l =  Log_transform(X_train)
X_trl, X_ttl, y_trl, y_ttl = split_data(X_l,Y_train)
train_predict(X_trl, X_ttl, y_trl, y_ttl," Log transformation ") ### Log transformation 

#####################    Log transformation   #########################
[[808   0   0   0   2   0   5   0   1   0]
 [  0 901   2   0   1   1   1   1   1   1]
 [  1   5 821   2   7   0   1   2   7   0]
 [  0   3   5 899   0   9   0   5   9   7]
 [  1   0   1   0 819   1   4   2   0  11]
 [  1   1   0  13   1 672   8   0   2   4]
 [  2   1   0   0   0   1 777   0   4   0]
 [  0   2   5   0   6   0   0 867   3  10]
 [  1   6   2   5   4   3   0   2 810   2]
 [  1   1   1  10   3   1   0   8   4 809]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.98      0.99      0.99       909
           2       0.98      0.97      0.98       846
           3       0.97      0.96      0.96       937
           4       0.97      0.98      0.97       839
           5       0.98      0.96      0.97       702
           6       0.98      0.99      0.98       785
           7       0.98      0.97      0.97       893
          

In [15]:
X_N =  mean_normalization(X_train)
X_trn, X_ttn, y_trn, y_ttn = split_data(X_N,Y_train)
train_predict(X_trn, X_ttn, y_trn, y_ttn," mean normalization  ") ### mean normalization

#####################    mean normalization    #########################
[[807   0   0   0   2   1   4   0   1   1]
 [  0 902   1   1   1   1   0   1   1   1]
 [  3   3 820   2   8   0   3   3   4   0]
 [  1   3   5 898   0   9   0   6  11   4]
 [  1   0   1   0 815   1   5   3   0  13]
 [  1   1   1   8   2 679   6   0   2   2]
 [  3   1   0   0   0   1 777   0   3   0]
 [  0   2   7   0   4   1   0 867   2  10]
 [  1   2   3   7   5   4   0   2 810   1]
 [  2   2   1  10  10   1   0   7   3 802]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.98      0.99      0.99       909
           2       0.98      0.97      0.97       846
           3       0.97      0.96      0.96       937
           4       0.96      0.97      0.97       839
           5       0.97      0.97      0.97       702
           6       0.98      0.99      0.98       785
           7       0.98      0.97      0.97       893
         

In [16]:
X_s =  zScore_Standarisation(X_train)
X_trs, X_tts, y_trs, y_tts = split_data(X_s,Y_train)
train_predict(X_trs, X_tts, y_trs, y_tts," zscore standarization  ") ### z-score standarisation

#####################    zscore standarization    #########################
[[801   0   3   0   1   3   7   0   1   0]
 [  0 892   9   1   1   1   1   1   2   1]
 [  2   6 820   3   4   0   4   2   5   0]
 [  1   3  12 884   0  11   0   9  12   5]
 [  1   0  11   0 805   1   5   4   0  12]
 [  2   1   8  14   1 663  10   1   2   0]
 [  5   1  11   0   0   6 760   0   2   0]
 [  0   2  11   3   5   2   0 855   1  14]
 [  1   4   9  10   4   8   4   2 791   2]
 [  2   2   6  12   7   3   0  19   4 783]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       816
           1       0.98      0.98      0.98       909
           2       0.91      0.97      0.94       846
           3       0.95      0.94      0.95       937
           4       0.97      0.96      0.97       839
           5       0.95      0.94      0.95       702
           6       0.96      0.97      0.96       785
           7       0.96      0.96      0.96       893
      

In [17]:
X_ll =  Log_transform(X_train)
X_ls =  zScore_Standarisation(X_ll)
X_trls, X_ttls, y_trls, y_ttls = split_data(X_ls,Y_train)
train_predict(X_trls, X_ttls, y_trls, y_ttls,"TL + Stand  ") ### TL + z-score standarisation

#####################   TL + Stand    #########################
[[805   0   1   1   1   1   5   1   1   0]
 [  0 890   3   1   1   1   2   7   3   1]
 [  1   5 814   4   3   1   3   6   7   2]
 [  1   3   9 884   0  10   0  10  13   7]
 [  1   0   6   0 808   1   5   5   0  13]
 [  2   1   0  16   2 662  11   6   2   0]
 [  3   1   0   0   0   5 765   9   2   0]
 [  0   3   7   6   6   1   0 856   0  14]
 [  1   7   5   7   4   9   1   7 792   2]
 [  2   1   2  13   4   2   0  16   4 794]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.98      0.98      0.98       909
           2       0.96      0.96      0.96       846
           3       0.95      0.94      0.95       937
           4       0.97      0.96      0.97       839
           5       0.96      0.94      0.95       702
           6       0.97      0.97      0.97       785
           7       0.93      0.96      0.94       893
           8      

In [19]:
X_lll =  Log_transform(X_train)
X_lm =  max_normalization(X_lll)
X_trlm, X_ttlm, y_trlm, y_ttlm = split_data(X_lm,Y_train)
train_predict(X_trlm, X_ttlm, y_trlm, y_ttlm,"TL + max noraml  ") ### TL + max normalization

#####################   TL + min noraml    #########################
[[808   0   0   0   2   0   5   0   1   0]
 [  0 901   2   0   1   1   1   1   1   1]
 [  1   5 821   2   7   0   1   2   7   0]
 [  0   3   5 899   0   9   0   5   9   7]
 [  1   0   1   0 819   1   4   2   0  11]
 [  1   1   0  13   1 672   8   0   2   4]
 [  2   1   0   0   0   1 777   0   4   0]
 [  0   2   5   0   6   0   0 867   3  10]
 [  1   6   2   5   4   3   0   2 810   2]
 [  1   1   1  10   3   1   0   8   4 809]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.98      0.99      0.99       909
           2       0.98      0.97      0.98       846
           3       0.97      0.96      0.96       937
           4       0.97      0.98      0.97       839
           5       0.98      0.96      0.97       702
           6       0.98      0.99      0.98       785
           7       0.98      0.97      0.97       893
           8 

In [21]:
X_pca95 =  applyPCA (X_train,p=.95)
X_tr95, X_tt95, y_tr95, y_ttp95 = split_data(X_pca95,Y_train)
train_predict(X_tr95, X_tt95, y_tr95, y_ttp95," PCA .95 ") ### PCA .95

#####################    PCA .95   #########################
[[808   0   0   0   2   0   4   0   1   1]
 [  0 903   1   0   1   1   0   1   1   1]
 [  2   3 824   2   7   0   2   2   4   0]
 [  0   1   4 905   0   9   0   4  10   4]
 [  1   0   1   0 818   1   4   2   0  12]
 [  1   1   1   7   2 682   4   0   2   2]
 [  3   1   0   0   0   1 777   0   3   0]
 [  0   2   5   0   4   1   0 871   2   8]
 [  1   2   3   3   3   3   0   2 817   1]
 [  2   1   1   9   7   2   0   6   3 807]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.99      0.99      0.99       909
           2       0.98      0.97      0.98       846
           3       0.98      0.97      0.97       937
           4       0.97      0.97      0.97       839
           5       0.97      0.97      0.97       702
           6       0.98      0.99      0.99       785
           7       0.98      0.98      0.98       893
           8       0.

In [22]:
X_pca10 =  applyPCA (X_train,p=.10)
X_tr10, X_tt10, y_tr10, y_ttp10 = split_data(X_pca10,Y_train)
train_predict(X_tr10, X_tt10, y_tr10, y_ttp10," PCA .10 ") ### PCA .10

#####################    PCA .10   #########################
[[627   0  41  14   1  15 105   0  12   1]
 [  0 867   3   1   0  13   1   4  20   0]
 [107  23 188 146  36  36 137   8 165   0]
 [ 28  18  61 472  17  47  60   9 223   2]
 [  1  11   0   0 506  17  54 132   6 112]
 [ 58  16 101  93  59  90 141   7 136   1]
 [ 93  15  52  20  67  25 352   1 159   1]
 [  0  38   0   1 269  17  28 420   9 111]
 [ 50  19  53 122  38  54 161   6 332   0]
 [  6  15   4   1 360  12  43 236  10 151]]
              precision    recall  f1-score   support

           0       0.65      0.77      0.70       816
           1       0.85      0.95      0.90       909
           2       0.37      0.22      0.28       846
           3       0.54      0.50      0.52       937
           4       0.37      0.60      0.46       839
           5       0.28      0.13      0.18       702
           6       0.33      0.45      0.38       785
           7       0.51      0.47      0.49       893
           8       0.

In [23]:
X_chi2 = chi_square(X_train,Y_train,nbf=200)
X_trchi, X_ttchi, y_trchi, y_ttpchi = split_data(X_chi2,Y_train)
train_predict(X_trchi, X_ttchi, y_trchi, y_ttpchi," Chi 2  ") ### CHI2

#####################    Chi 2    #########################
[[807   0   1   0   2   1   5   0   0   0]
 [  0 901   1   0   1   1   2   1   2   0]
 [  3   2 809   3   5   3   3  12   5   1]
 [  0   3   6 893   1  18   0   8   7   1]
 [  2   2   0   0 771   0   7   1   3  53]
 [  1   1   2  17   1 668   7   0   2   3]
 [  4   1   2   0   0   3 772   0   3   0]
 [  0   7   8   1  13   1   0 857   1   5]
 [  1   6   4   2   3   3   5   3 805   3]
 [  2   3   1   8  20   5   1   9   4 785]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       816
           1       0.97      0.99      0.98       909
           2       0.97      0.96      0.96       846
           3       0.97      0.95      0.96       937
           4       0.94      0.92      0.93       839
           5       0.95      0.95      0.95       702
           6       0.96      0.98      0.97       785
           7       0.96      0.96      0.96       893
           8       0.9

In [24]:
X_relief100 = apply_relief_selection(X_train,Y_train,nbf=100)
X_rlf100, X_lrf100, y_lrf100, y_frf100 = split_data(X_relief100,Y_train)
train_predict(X_rlf100, X_lrf100, y_lrf100, y_frf100," RELIF 100  ") ###  RELIEF 100

#####################    RELIF 100    #########################
[[794   0   4   2   0   9   1   4   1   1]
 [  0 902   3   0   1   0   2   1   0   0]
 [  7   8 778   7  10   7   5  12   3   9]
 [  3   1  25 858   1  14   1   7  13  14]
 [  1   1   1   0 804   0   9   3   3  17]
 [  4   2   0  10   2 663   9   4   5   3]
 [  3   1   1   2   4  10 761   0   3   0]
 [  5   2  18   6   5   0   1 842   1  13]
 [  2   2   8  18   4  13   0   4 777   7]
 [  3   2   9   7   6   9   0  14   7 781]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       816
           1       0.98      0.99      0.99       909
           2       0.92      0.92      0.92       846
           3       0.94      0.92      0.93       937
           4       0.96      0.96      0.96       839
           5       0.91      0.94      0.93       702
           6       0.96      0.97      0.97       785
           7       0.95      0.94      0.94       893
           8      

In [25]:
X_relief10 = apply_relief_selection(X_train,Y_train,nbf=10)
X_rlf10, X_lrf10, y_lrf10, y_frf10 = split_data(X_relief10,Y_train)
train_predict(X_rlf10, X_lrf10, y_lrf10, y_frf10," RELief 10 ") ###  RELIEF 10

#####################    RELief 10   #########################
[[459   8  34  36  10 130  54  35  13  37]
 [  1 661   4   9  81   2 139   8   1   3]
 [ 88  60 204 139  78  68  50  27  55  77]
 [ 52 102  75 378  24  28  51  33 104  90]
 [  4  94  22   7 558   2  77  29  18  28]
 [113   8   6   9  16 340  83  87  28  12]
 [ 25  82   1   4  76  26 517  38   7   9]
 [ 24  23   5  21  47  73  20 510  13 157]
 [109  18  53 148  32  95  48  23 216  93]
 [ 72  14  11  52  23  64  23  99  58 422]]
              precision    recall  f1-score   support

           0       0.48      0.56      0.52       816
           1       0.62      0.73      0.67       909
           2       0.49      0.24      0.32       846
           3       0.47      0.40      0.43       937
           4       0.59      0.67      0.63       839
           5       0.41      0.48      0.44       702
           6       0.49      0.66      0.56       785
           7       0.57      0.57      0.57       893
           8       

In [38]:
X_ref100 = apply_rfe(X_train,Y_train,threshold = 100)

X_train_f100, X_test_f100, y_train_f100, y_test_f100 = split_data(X_ref100,Y_train)
train_predict(X_train_f100, X_test_f100, y_train_f100, y_test_f100, "REF 100")  #### REF 100

KeyboardInterrupt: 

In [ ]:
X_ref10 = apply_rfe(X_train,Y_train,threshold = 10)

X_train_f10, X_test_f10, y_train_f10, y_test_f10 = split_data(X_ref10,Y_train)
train_predict(X_train_f10, X_test_f10, y_train_f10, y_test_f10, "REF 10")  #### REF 10

In [41]:
## best performance 
#X_pca95 =  applyPCA(X_train,p=.95)

from sklearn.svm import SVC

svclassifier = SVC(kernel='rbf') #0.9591836734693877
t0 = time()
svclassifier.fit(X_train, Y_train) 
#X_pca95.shape

C:\Users\X1\anaconda3\envs\deepl\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [42]:
X_train.shape

(42000, 784)

In [43]:
X_test.shape

(28000, 784)

In [44]:
#X_testpca =  applyPCA (X_test,p=.95)
#X_testpca.shape


In [45]:

y_pred = svclassifier.predict(X_test)
id = range(1,28001)
submission = pd.DataFrame(list(zip(id, y_pred)),columns =['ImageId', 'Label'])
submission.head(20)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [46]:
submission.to_csv('C:/Users/X1/Desktop/master - S3/image-mining/ImageMining/my_submission.csv', index=False)